---
# 환경 설정
---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mxnet
!pip install gluonnlp tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 277 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 7.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595738 sha256=4dcd6016bc13a638b6dd7072c6eb541aec854fadf2cb978ff70c5111bce75f1f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 8.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [3]:
# !git clone https://github.com/SKTBrain/KoBERT.git

In [4]:
cd /content/drive/MyDrive/aiffel/NLP/KoBERT

/content/drive/MyDrive/aiffel/NLP/KoBERT


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 7.2 MB/s 
     |████████████████████████████████| 54.7 MB 39 kB/s 
     |████████████████████████████████| 4.5 MB 75.5 MB/s 
     |████████████████████████████████| 1.2 MB 66.9 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:44tcmalloc: large alloc 1147494400 bytes == 0x39f0e000 @  0x7f250f921615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 16 kB/s 
     |████████████████████████████████| 2.5 MB 60.0 MB/s 
     |████████████████████████████████| 73 kB 2.2 MB/s 
     |████████████████████████████████| 6.7 MB 66.9 MB/s 
     |█████████████████████████

In [6]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 7.9 MB/s 


In [7]:
# from torchmetrics import F1Score

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [9]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [10]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [11]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


---
# 학습용 데이터셋 전처리
---

In [13]:
data2 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/test_0822.csv', encoding = 'cp949')

In [14]:
del data2['Column1']


In [39]:
data1 = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/test.csv')

In [40]:
data1.columns = ['1','review','label']
data1

,1,review,label
0,33311,치킨은 항상 비비큐에요,1
1,33312,정말 친절하고 맛있습니다,"1,2"
2,33313,괜찮아요 빨리왔어요,3
3,33314,순살만들어주새요그럼개사머음,4
4,33315,프라이드 치킨이 있었으면 좋겠어요,4
...,...,...,...
12766,46144,잘 먹었습니다 서비스로 날치알밥도 맛있게 잘 먹었어요,NaN
12767,46145,항상 여기서 시켜먹어요 다 좋아요,NaN
12768,46146,가성비 최고 맛도 웬만한가게보다 훨씬나아요 앞으로 자주먹을듯,NaN
12769,46147,배달빠르고 맛보기 무침족발 새콤달콤하니 맛있어요,NaN


In [41]:
del data1['1']

In [18]:
data = pd.concat([data1,data2],axis = 0, ignore_index = True)
data

,review,label
0,치킨은 항상 비비큐에요,1
1,정말 친절하고 맛있습니다,"1,2"
2,괜찮아요 빨리왔어요,3
3,순살만들어주새요그럼개사머음,4
4,프라이드 치킨이 있었으면 좋겠어요,4
...,...,...
25710,할인까지 받아서 정말 맛나게 머것어요,NaN
25711,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,NaN
25712,황금올리브 닭다리는 언제나 옳아요 맛있어요,NaN
25713,맛도좋구배달도빠르네요,NaN


In [19]:
data = data.dropna()

In [20]:
data[['taste','quantity','delivery','else']] = np.NaN

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3645: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = value


In [21]:
data['label'] = data['label'].apply(lambda x : x.split(','))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
data = data.reset_index(drop = True)

In [23]:
for i in range(len(data)):
  if '1' in data['label'][i]:
    data['taste'][i] = 1
  else :
    data['taste'][i] = 0

  if '2' in data['label'][i]:
    data['quantity'][i] = 1
  else :
    data['quantity'][i] = 0

  if '3' in data['label'][i]:
    data['delivery'][i] = 1
  else :
    data['delivery'][i] = 0

  if '4' in data['label'][i]:
    data['else'][i] = 1
  else :
    data['else'][i] = 0        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [24]:
del data['label']

In [25]:
data[['taste','quantity','delivery','else']] = data[['taste','quantity','delivery','else']].astype(int)

In [26]:
data_list = [] # 최종 데이터셋 리스트 
for q, taste_label, quantity_label, delivery_label in zip(data['review'], data['taste'], data['quantity'], data['delivery']) :
  dataset = []  # 라벨과 내용이 하나의 리스트에 넣기 위해서 빈 리스트 생성
  dataset.append(q)
  dataset.append(str(taste_label))
  dataset.append(str(quantity_label))
  dataset.append(str(delivery_label))

  data_list.append(dataset)

In [27]:
data_list[:3]

[['치킨은 항상 비비큐에요', '1', '0', '0'],
 ['정말 친절하고 맛있습니다', '1', '1', '0'],
 ['괜찮아요 빨리왔어요', '0', '0', '1']]

In [28]:
import numpy as np 

total_data_text = list(data['review'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  28.767739602169982
문장길이 최대 :  99
문장길이 표준편차 :  19.44976040679062
pad_sequences maxlen :  67
전체 문장의 0.9358408679927668%가 maxlen 설정값 이내에 포함됩니다. 


In [29]:
train_data, test_data = train_test_split(data_list, test_size = 0.1, random_state = 53)

---
# 모델
---

In [13]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length = max_len, pad = pad, pair = pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.taste = [np.int64(i[label_idx]) for i in dataset]
    self.quantity = [np.int64(i[label_idx+1]) for i in dataset]
    self.delivery = [np.int64(i[label_idx+2]) for i in dataset]

  def __getitem__(self,i):
    return (self.sentences[i] + (self.taste[i],self.quantity[i],self.delivery[i] ))

  def __len__(self):
    return (len(self.taste))

In [14]:
# 하이퍼파라미터 세팅
max_len = 65
batch_size = 16
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 400
learning_rate =  4e-5

In [15]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False ,num_best = -1 )

using cached model. /content/drive/MyDrive/aiffel/NLP/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [33]:
data_train = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_data, 0, 1, tok, max_len, True, False)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=False, batch_size=batch_size, num_workers=2, drop_last=True)

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=1,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.taste = nn.Linear(hidden_size , num_classes)
        self.quantity = nn.Linear(hidden_size , num_classes)
        self.delivery = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

        self.sigmoid = nn.Sigmoid()
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, out = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(out)

        taste_out = self.taste(out)
        quantity_out = self.quantity(out)
        delivery_out = self.delivery(out)   

        return self.sigmoid(taste_out), self.sigmoid(quantity_out), self.sigmoid(delivery_out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.BCELoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [37]:
for i in train_dataloader:
  print(i)
  break

[tensor([[   2, 2287, 6398,  ...,    1,    1,    1],
        [   2,  517, 6153,  ...,    1,    1,    1],
        [   2, 4128, 6079,  ...,    1,    1,    1],
        ...,
        [   2, 2705, 7318,  ...,    1,    1,    1],
        [   2, 3214, 5859,  ...,    1,    1,    1],
        [   2, 1967, 7141,  ...,    1,    1,    1]], dtype=torch.int32), tensor([ 9,  8, 22, 24, 16, 18, 25, 21, 37, 11, 26, 16, 17,  8,  8, 11],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]), tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [38]:

for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length

        taste_label = taste_label.float().to(device)
        taste_label = taste_label.unsqueeze(1)

        quantity_label = quantity_label.float().to(device)
        quantity_label = quantity_label.unsqueeze(1)

        delivery_label = delivery_label.float().to(device)
        delivery_label = delivery_label.unsqueeze(1)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)
         
        # Loss
        ################################################################################################
        taste_loss = loss_fn(taste_out, taste_label)
        quantity_loss = loss_fn(quantity_out, quantity_label)
        delivery_loss = loss_fn(delivery_out, delivery_label)

        total_loss = (taste_loss + quantity_loss + delivery_loss)
        total_loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # taste_accuracy += calc_accuracy(taste_out, taste_label)
        # quantity_accuracy += calc_accuracy(quantity_out, quantity_label)
        # delivery_accuracy += calc_accuracy(delivery_out, delivery_label)

        # Accuracy
        ################################################################################################
        taste_preds = torch.argmax(taste_out, dim=1).flatten()
        quantity_preds = torch.argmax(quantity_out, dim=1).flatten()
        delivery_preds = torch.argmax(delivery_out, dim=1).flatten()

        taste_accuracy = (taste_preds == taste_label).cpu().numpy().mean()
        quantity_accuracy = (quantity_preds == quantity_label).cpu().numpy().mean()
        delivery_accuracy = (delivery_preds == delivery_label).cpu().numpy().mean()

        train_acc += (taste_accuracy + quantity_accuracy + delivery_accuracy) / 3
        ################################################################################################
        
        # F1_Score
        ################################################################################################
        taste_f1 = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'weighted')
        quantity_f1 = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'weighted')
        delivery_f1 = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'weighted')
        
        total_f1 +=  (taste_f1 + quantity_f1 + delivery_f1) / 3
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {total_loss.data.cpu().numpy():.3f} train acc {train_acc / (batch_id+1):.3f}")
            print(f'taste_accuracy {taste_accuracy:.3f}  | quantity_accuracy {quantity_accuracy:.3f}  | delivery_accuracy {delivery_accuracy:.3f}')
            print(f'f1_score:{total_f1 / (batch_id+1):.3f}','\n')
            print('#'*80)
    print(f"epoch {e+1} train acc {train_acc / (batch_id+1):.3f}, f1_score {total_f1 / (batch_id+1):.3f}")
    # train_history.append(train_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/777 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.290 train acc 0.562
taste_accuracy 0.250  | quantity_accuracy 0.750  | delivery_accuracy 0.688
f1_score:0.691 

################################################################################
epoch 1 batch id 401 loss 0.841 train acc 0.581
taste_accuracy 0.125  | quantity_accuracy 0.875  | delivery_accuracy 0.750
f1_score:0.695 

################################################################################
epoch 1 train acc 0.581, f1_score 0.695


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/777 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.612 train acc 0.604
taste_accuracy 0.188  | quantity_accuracy 0.875  | delivery_accuracy 0.750
f1_score:0.702 

################################################################################
epoch 2 batch id 401 loss 0.216 train acc 0.578
taste_accuracy 0.312  | quantity_accuracy 0.812  | delivery_accuracy 0.750
f1_score:0.693 

################################################################################
epoch 2 train acc 0.581, f1_score 0.695


In [39]:
model.eval()
test_f1 = 0.0
test_acc = 0.0
for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length

    taste_label = taste_label.float().to(device)
    taste_label = taste_label.unsqueeze(1)

    quantity_label = quantity_label.float().to(device)
    quantity_label = quantity_label.unsqueeze(1)

    delivery_label = delivery_label.float().to(device)
    delivery_label = delivery_label.unsqueeze(1)

    taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

    # Accuracy
    ################################################################################################    
    taste_preds_test = torch.argmax(taste_out, dim=1).flatten()
    quantity_preds_test = torch.argmax(quantity_out, dim=1).flatten()
    delivery_preds_test = torch.argmax(delivery_out, dim=1).flatten()

    taste_accuracy_test = (taste_preds_test == taste_label).cpu().numpy().mean()
    quantity_accuracy_test = (quantity_preds_test == quantity_label).cpu().numpy().mean()
    delivery_accuracy_test = (delivery_preds_test == delivery_label).cpu().numpy().mean()

    test_acc += (taste_accuracy_test + quantity_accuracy_test + delivery_accuracy_test) / 3
    ################################################################################################ 

    # F1_Score
    ################################################################################################
    taste_f1_test = f1_score(taste_preds.cpu().data, taste_label.cpu(), average = 'weighted')
    quantity_f1_test = f1_score(quantity_preds.cpu().data, quantity_label.cpu(), average = 'weighted')
    delivery_f1_test = f1_score(delivery_preds.cpu().data, delivery_label.cpu(), average = 'weighted')
    
    test_f1 +=  ((taste_f1_test + quantity_f1_test + delivery_f1_test) / 3)
    ################################################################################################

print(f"train acc {test_acc / (batch_id+1):.3f}, f1_score {test_f1/ (batch_id+1):.3f}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/86 [00:00<?, ?it/s]

train acc 0.576, f1_score 0.691


In [96]:
# torch.save(model, '/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0825_first.pt')

In [18]:
model = torch.load('/content/drive/MyDrive/aiffel/AIFFELTON/model_save/koBERT_Multi_label_0825_first.pt')

In [19]:
def predict(predict_sentence):

    data = [predict_sentence, '0', '0','0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

        for taste, quantity, delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = taste
            logits_quantity = quantity
            logits_delivery = delivery

            logits_taste = logits_taste.detach().cpu().numpy()
            logits_quantity = logits_quantity.detach().cpu().numpy()
            logits_delivery = logits_delivery.detach().cpu().numpy()

            print('맛에 대한 점수:',logits_taste)
            print('양 or 서비스에 대한 점수:',logits_quantity)
            print('배달에 대한 점수:',logits_delivery)

---
# 결과값 테스트
---

In [21]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
predict('맛이 없다')

맛에 대한 점수: [0.99448544]
양 or 서비스에 대한 점수: [0.00825016]
배달에 대한 점수: [0.01003474]


In [23]:
predict('서비스가 좋아서 다시 가고 싶어요')

맛에 대한 점수: [0.03820682]
양 or 서비스에 대한 점수: [0.9729021]
배달에 대한 점수: [0.23614635]


In [45]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

맛에 대한 점수: [0.42349845]
양 or 서비스에 대한 점수: [0.986291]
배달에 대한 점수: [0.04062264]


In [46]:
predict('음식은 겉바속촉 매장은 깔끔')

맛에 대한 점수: [0.1139733]
양 or 서비스에 대한 점수: [0.0176501]
배달에 대한 점수: [0.36256778]


In [47]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

맛에 대한 점수: [0.91373247]
양 or 서비스에 대한 점수: [0.0131265]
배달에 대한 점수: [0.9743558]


In [48]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

맛에 대한 점수: [0.01430795]
양 or 서비스에 대한 점수: [0.6782083]
배달에 대한 점수: [0.87935436]


In [49]:
predict('맛 양 배달 모두 만족합니다')

맛에 대한 점수: [0.92994595]
양 or 서비스에 대한 점수: [0.91690993]
배달에 대한 점수: [0.95615107]


In [50]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

맛에 대한 점수: [0.0126913]
양 or 서비스에 대한 점수: [0.8620576]
배달에 대한 점수: [0.7591773]


In [51]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

맛에 대한 점수: [0.0366987]
양 or 서비스에 대한 점수: [0.02394924]
배달에 대한 점수: [0.97225857]


In [52]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

맛에 대한 점수: [0.077684]
양 or 서비스에 대한 점수: [0.00897605]
배달에 대한 점수: [0.9281383]


In [53]:
predict('사업 번창하세요')

맛에 대한 점수: [0.37784842]
양 or 서비스에 대한 점수: [0.01893879]
배달에 대한 점수: [0.03601123]


In [54]:
predict('다른집 들렸다 온것 같아요')

맛에 대한 점수: [0.06808319]
양 or 서비스에 대한 점수: [0.01211599]
배달에 대한 점수: [0.76626116]


In [24]:
predict('다른집 들렸다 온것 같아요')

맛에 대한 점수: [0.06808319]
양 or 서비스에 대한 점수: [0.01211599]
배달에 대한 점수: [0.76626116]


----
# 비라벨링 -> 모델로 라벨링 
---

In [25]:
labeling = pd.read_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/model_labeling_0826_young_woking.csv')
# labeling.columns = ['1','review','label']
labeling

,Column1,review,label
0,28473,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다,NaN
1,28474,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ,NaN
2,28475,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지,NaN
3,28476,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...,NaN
4,28477,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요,NaN
...,...,...,...
4501,32996,할인까지 받아서 정말 맛나게 머것어요,NaN
4502,32997,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,NaN
4503,32998,황금올리브 닭다리는 언제나 옳아요 맛있어요,NaN
4504,32999,맛도좋구배달도빠르네요,NaN


In [26]:
del labeling['Column1']

In [27]:
labeling

,review,label
0,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다,NaN
1,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ,NaN
2,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지,NaN
3,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...,NaN
4,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요,NaN
...,...,...
4501,할인까지 받아서 정말 맛나게 머것어요,NaN
4502,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,NaN
4503,황금올리브 닭다리는 언제나 옳아요 맛있어요,NaN
4504,맛도좋구배달도빠르네요,NaN


In [28]:
label_list = []
for i in range(len(labeling)):
  label_list.append(labeling['review'][i])

In [30]:
def predict(predict_sentence):

    data = [predict_sentence, '0', '0','0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, taste_label, quantity_label, delivery_label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length

        taste_label = taste_label.long().to(device)
        quantity_label = quantity_label.long().to(device)
        delivery_label = delivery_label.long().to(device)

        taste_out, quantity_out, delivery_out = model(token_ids, valid_length, segment_ids)

        for logits_taste, logits_quantity, logits_delivery in zip(taste_out, quantity_out, delivery_out):

            logits_taste = logits_taste.detach().cpu().numpy()
            logits_quantity = logits_quantity.detach().cpu().numpy()
            logits_delivery = logits_delivery.detach().cpu().numpy()

    labeling = []
    if logits_taste >= 0.55:
      labeling.append('1')
    else :
      labeling.append('0')

    if logits_quantity >= 0.55:
      labeling.append('1')
    else :
      labeling.append('0')

    if logits_delivery >= 0.55:
      labeling.append('1')
    else :
      labeling.append('0')
    
    return labeling

In [31]:
multl_label = []
for corpus in label_list:
  value = predict(corpus)
  multl_label.append(value)
  

In [34]:
a = pd.DataFrame(label_list)
a.columns = ['review']
a

,review
0,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다
1,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ
2,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지
3,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...
4,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요
...,...
4501,할인까지 받아서 정말 맛나게 머것어요
4502,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요
4503,황금올리브 닭다리는 언제나 옳아요 맛있어요
4504,맛도좋구배달도빠르네요


In [35]:
b = pd.DataFrame(multl_label)
b.columns = ['taste','quantity','delivery']
b

,taste,quantity,delivery
0,1,1,1
1,1,0,0
2,1,0,1
3,1,1,0
4,1,0,0
...,...,...,...
4501,1,0,0
4502,1,0,0
4503,1,0,0
4504,1,0,1


In [36]:
b['review'] = a['review']

In [37]:
b = b.reindex( columns = ['review','taste','quantity','delivery'])
b

,review,taste,quantity,delivery
0,예정시간에 맞춰서 배달되었고 양도 많았어요 맛있게 잘먹었습니다,1,1,1
1,요기요에서는 처음 주문 해보네요 역시나 맛있습니다 ㅎㅎㅎ,1,0,0
2,배달은 예상시간보다 빨리와야 제맛 간장찜닭도 맛잇고 김치찜닭도 맛잇고 담엔 둘중에 뭐먹지,1,0,1
3,닭이 순살로 와서 먹기 편했어요 살짝 짜고 매웠는데 매식이 다 그렇죠 양 겁나 많아...,1,1,0
4,보통맛이지만 매운거 못먹는 사람들은 매울 수 있으니 순한맛 추천드려요,1,0,0
...,...,...,...,...
4501,할인까지 받아서 정말 맛나게 머것어요,1,0,0
4502,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요,1,0,0
4503,황금올리브 닭다리는 언제나 옳아요 맛있어요,1,0,0
4504,맛도좋구배달도빠르네요,1,0,1


In [38]:
b.to_csv('/content/drive/MyDrive/aiffel/AIFFELTON/for_model_labeling_folder/check_young_0826.csv')